<a href="https://colab.research.google.com/github/arunabh-alt/CI-CD_Pipeline_Practice/blob/main/ELT_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade opentelemetry-sdk opentelemetry-api


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.37b0
    Uninstalling opentelemetry-semantic-conventions-0.37b0:
      Successfully uninstalled opentelemetry-semantic-conventions-0.37b0
  Attempting uninstall: opentelemetry-sdk
    Found existing installation: opentelemetry-sdk 1.16.0
    Uninstalling opentelemetry-sdk-1.16.0:
      Successfully uninstalled opentelemetry-sdk-1.16.0


In [5]:
# Install necessary packages
!pip install --quiet prefect sqlalchemy pandas matplotlib requests


In [7]:
!pip install "prefect<2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.6/606.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: prefect
    Found existing installation: prefect 3.4.0
    Uninstalling prefect-3.4.0:
      Successfully uninstalled prefect-3.4.0


In [1]:
import sqlite3
import pandas as pd
import requests
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from prefect import task, Flow
from prefect.schedules import IntervalSchedule
from datetime import timedelta

Extract

In [2]:
@task
def fetch_data(country_slug : str) -> pd.DataFrame:
  url = f"https://api.covid19api.com/dayone/country/{country_slug}"
  resp = requests.get(url)
  resp.raise_for_status()
  data = resp.join()
  df = pd.json_normalize(data)

  return df[ ["Country", "CountryCode", "Province", "City", "CityCode",
        "Lat", "Lon", "Confirmed", "Deaths", "Recovered", "Active", "Date"]]

Load

In [9]:
DB_URI = "sqlite:///covid_data.db"
engine = create_engine(DB_URI, echo= False)

@task
def load_raw(df: pd.DataFrame):

    df.to_sql("raw_cases", engine, if_exists="append", index=False)

Transform

1. Build daily totals with SQL

In [4]:
@task
def transform_daily():
  sql = """
  CREATE TABLE IF NOT EXISTS daily_total AS
  SELECT
    DATE(Date) AS day,
    SUM(Confirmed) AS total_confirmed,
    SUM(Deaths) AS total_deaths,
    SUM(Recovered) AS total_recovered,
  FROM raw_cases
  GROUP BY day
  ORDER BY day;
  """
  with engine.begin() as conn:
    conn.execute(sql)


2. Compute 7-day moving average with pandas

In [5]:
@task
def transform_moving_avg() -> pd.DataFrame:
  df = pd.read_sql("SELECT * FROM daily_totals", engine, parse_dates = ["day"])
  df = df.sort_values("day")
  df["new_confirmed"] = df["total_confirmed"].diff().fillna(0)
  df["ma_7"] = df["new_confirmed"].rolling(window = 7).mean()

  df.to_sql("moving_avg", engine, if_exists="replace", index=False)
  return df

Visualize

In [6]:
@task
def plot_trends(df: pd.DataFrame):
    plt.figure(figsize=(10,4))
    plt.plot(df["day"], df["new_confirmed"], label="Daily New Cases")
    plt.plot(df["day"], df["ma_7"], label="7‑Day MA")
    plt.title("COVID‑19 Daily New Cases & 7‑Day MA")
    plt.xlabel("Date")
    plt.ylabel("Cases")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
schedule = IntervalSchedule(interval= timedelta(days=1))

with Flow("covid-elt-pipeline", schedule = schedule) as flow:
  raw_df = fetch_data(country_slug="china")
  _  = load_raw(raw_df)
  _  = transform_daily()
  mov_avg_df = transform_moving_avg()
  _ = plot_trends(mov_avg_df)

flow.run()

[2025-05-07 13:48:20+0000] INFO - prefect.covid-elt-pipeline | Waiting for next scheduled run at 2025-05-08 00:00:00+00:00


INFO:prefect.covid-elt-pipeline:Waiting for next scheduled run at 2025-05-08 00:00:00+00:00
